### Load packages

In [1]:
import os, sys
import numpy as np
import pandas as pd
from glob import glob 
from skimage.io import imread, imsave
import zarr
import napari

### Start Napari viewer

In [2]:
%gui qt
napari.gui_qt()
viewer = napari.Viewer(ndisplay=3)

### Specify directory to data files

In [3]:
# roi_dir='test-oct6_epsilon1\\roi.csv'
# lb_dir   ="/Users/fangmingxie/data/easifish/lt185/r1/segmentation/r1-c3.tif"
# spot_dir ="/Users/fangmingxie/data/easifish/lt185_r1_redo/spots/spots_c0.txt"
imdir    ="/Users/fangmingxie/data/easifish/lt186/r1/stitching/export.n5"

In [4]:
# lb = imread(lb_dir)
# lb.shape

In [5]:
# lb_sub = lb[:,400:450,:]
# lb_sub = np.swapaxes(lb_sub,0,1)

In [6]:
#viewer.add_labels(lb_sub.astype('uint32'),name='labels',blending='additive')

In [7]:
# get n5 image data for channel 1
im = zarr.open(store=zarr.N5Store(imdir), mode='r')
_img = im['c3/s4']
zdim, ydim, xdim = _img.shape
zdim, ydim, xdim

(288, 456, 565)

In [8]:
range_min, range_max = 0, ydim

In [9]:
img1 = _img[:,range_min:range_max,:]
print(img1.shape)
# img1 = np.swapaxes(img1,0,1)
# print(img1.shape)
viewer.add_image(img1,colormap='gray',blending='additive')

(288, 456, 565)


<Image layer 'img1' at 0x17a0195b0>

In [10]:
# from skimage.measure import block_reduce
# res = block_reduce(img1, block_size=(2,2,2), func=np.mean)
# res.shape

In [11]:
# get n5 image data for channel 1
_img = im['c0/s4']
zdim, ydim, xdim = _img.shape
img1 = _img[:,range_min:range_max,:]
print(img1.shape)
# img1 = np.swapaxes(img1,0,1)
# print(img1.shape)

viewer.add_image(img1,colormap='blue',blending='additive', name='RL (Cre)')

(288, 456, 565)


<Image layer 'RL (Cre)' at 0x17aae4be0>

In [12]:
# get n5 image data for channel 1
_img = im['c2/s4']
zdim, ydim, xdim = _img.shape
img1 = _img[:,range_min:range_max,:]
print(img1.shape)
# img1 = np.swapaxes(img1,0,1)
# print(img1.shape)

viewer.add_image(img1,colormap='red',blending='additive', name='LM (FlpO)')

(288, 456, 565)


<Image layer 'LM (FlpO)' at 0x17ab333a0>

### Spot detection can also be visualized 

In [13]:
spot=np.loadtxt(spot_dir,delimiter=',')
# vox=np.array([0.23,0.23,0.42])*np.array([8,8,4])  #current image voxel size
vox=np.array([0.23,0.23,0.42])*np.array([16,16,9])  #current image voxel size
spot[:,:3]=spot[:,:3]/vox

In [14]:
_x = spot[:,0]
_y = spot[:,1]
_z = spot[:,2]
# x_min, y_min, z_min = 0,400,0
# x_max, y_max, z_max = xdim, 600, zdim
# conditions = [
#     _x <= x_max,
#     _y <= y_max,
#     _z <= z_max,
    
#     _x >  x_min,
#     _y >  y_min,
#     _z >  z_min,
# ]
# cond = (np.sum(conditions, axis=0) == 6) # all six conditions has to be satified to be pulled

# b = spot[cond]
b = spot
bx=b[:,0]-x_min
by=b[:,1]-y_min
bz=b[:,2]-z_min
point=np.stack((by,bz,bx),axis=-1)
viewer.add_points(point, size=1, face_color='red',edge_color='red',name='detected_spots')

NameError: name 'x_min' is not defined

### Specify ROI ID, block size and image channel 

In [ ]:
#Input information here: cell ID, image block size, channels
cell_id = 15 

#define the size of the block to display around the ROI
dx,dy,dz=[100,100,50]  # in pixel 
channel_1='c3'
channel_2='c0'

### Load data

In [ ]:
roi = pd.read_csv(roi_dir,sep=',', index_col=0) #, index_col=1)
roi

In [ ]:
# get n5 image data for channel 1
im = zarr.open(store=zarr.N5Store(imdir), mode='r')
_img = im[channel_1+'/s2']
zdim, ydim, xdim = _img.shape

In [ ]:
lb=imread(lb_dir)
z=roi.loc[cell_id,'z']*2/0.84 # convert to pixel 
y=roi.loc[cell_id,'y']*2/0.92 # convert to pixel 
x=roi.loc[cell_id,'x']*2/0.92 # convert to pixel 
Z=int(z)
Y=int(y)
X=int(x)

z_min, z_max = max(0, Z-dz), min(zdim, Z+dz)
y_min, y_max = max(0, Y-dy), min(ydim, Y+dy)
x_min, x_max = max(0, X-dx), min(xdim, X+dx)

#Crop image to show the target cell and surrounding area
lb_sub=lb[z_min:z_max, y_min:y_max, x_min:x_max]
#Target Cell ID
z0 = z - z_min
y0 = y - y_min
x0 = x - x_min
a = np.array([[z0, y0, x0]])
a

### Display segmentation mask, image data and the position of the specified ROI

In [ ]:
# get n5 image data for channel 1
im = zarr.open(store=zarr.N5Store(imdir), mode='r')
img_1 = im[channel_1+'/s2'][z_min:z_max, y_min:y_max, x_min:x_max]
viewer.add_image(img_1,colormap='green',blending='additive')

#get n5 image data for channel 2
if channel_2!='':
    im = zarr.open(store=zarr.N5Store(imdir), mode='r')   
    img_2 = im[channel_2+'/s2'][z_min:z_max, y_min:y_max, x_min:x_max]
    viewer.add_image(img_2.astype('float32'),colormap='gray',blending='additive')
    
# x, y, z are cell centroid location (in pixel)
zo = z - z_min
yo = y - y_min
xo = x - x_min

a = np.array([[zo, yo, xo]])

viewer.add_points(a, size=5, face_color='red',edge_color='red',name='cell_centroid_position')
viewer.add_labels(lb_sub.astype('uint32'),name='labels',blending='additive')

In [ ]:
f = 'D:\\SWAP\\Vincent\\lt172_exps\\gene\\outputs\\test-oct6_epsilon1\\clustering.csv'
res = pd.read_csv(f, index_col=0)
res

In [ ]:
len(roi), len(res)

In [ ]:
comm = np.intersect1d(roi.index.values, 
                      res.index.values,
                     )
comm.shape

In [ ]:
cluster_cols = ['clst_l1', 'clst_l2', 'clst_l3'] 
roi2 = roi.join(res[cluster_cols]).dropna()
roi2[cluster_cols] = roi2[cluster_cols].astype(int)  

roi2sub = roi2[roi2['clst_l1']==1]
_z = roi2sub['z'].values*2/0.84 # convert to pixel
_y = roi2sub['y'].values*2/0.92 # convert to pixel
_x = roi2sub['x'].values*2/0.92 # convert to pixel

conditions = [
    _x <= x_max,
    _y <= y_max,
    _z <= z_max,
    
    _x >  x_min,
    _y >  y_min,
    _z >  z_min,
]
cond = (np.sum(conditions, axis=0) == 6) # all six conditions has to be satified to be pulled
points = np.vstack([_z-z_min,
                    _y-y_min,
                    _x-x_min]).T[cond]

In [ ]:
# visualize the centroids of one cell type
viewer.add_points(points, size=10, face_color='blue',edge_color='blue',name='cluster 1')

In [ ]:
roi2['clst_l1'].unique()

In [ ]:
# visualize masks of all cell types (in diff colors)
lookup = roi2['clst_l1']
roiidx, inv = np.unique(lb_sub, return_inverse=True)
# np.all(lb_sub == roiidx[inv].reshape(lb_sub.shape))
clstlbs = lookup.reindex(roiidx).fillna(0).astype(int).values #.values
masks_byclsts = clstlbs[inv].reshape(lb_sub.shape)
viewer.add_labels(masks_byclsts.astype('uint32'), name='clusters',blending='additive')